In [2]:
import pandas as pd

In [95]:
df = pd.read_csv('owid-covid-data.csv')

df = df.groupby('iso_code').tail(1)

df['Country'] = df['location']

In [104]:
total_death_per_million = df.iloc[:,[1,9]]
total_death_per_million = total_death_per_million.fillna(0)
total_death_per_million.to_csv('TD_M.csv',index=False)

In [68]:
total = df.iloc[:,[1,3]]
total = total.fillna(0)

total.to_csv('total.csv',index=False)

In [85]:
TC_M = df.iloc[:,[0,7]]
TC_M = TC_M.fillna(0)
TC_M.to_csv('TC_M.csv',index=False)

In [108]:
import csv
import gviz_api

def get_page_template():
    page_template = """
    <html>
      <script src="https://www.google.com/jsapi" type="text/javascript"></script>
      <script>
        google.load('visualization', '1', {packages:['geochart', 'table']});

        google.setOnLoadCallback(drawMap);
        function drawMap() {
            var json_data = new google.visualization.DataTable(%s, 0.6);
            
            var options = {colorAxis: {colors: ['#eee', 'green']}};
            var mymap = new google.visualization.GeoChart(
                                document.getElementById('map_div'));
            mymap.draw(json_data, options);
            
            var mytable = new google.visualization.Table(
                                document.getElementById('table_div'));
            mytable.draw(json_data, {showRowNumber: true})
        }
      </script>
      <body>
        <H1>Covid19 Total Cases Per Million World Countries</H1>

        <div id="map_div"></div>
        <hr />
        <div id="table_div"></div>

        <div id="source">
        <hr />
        <small>
        Source: 
        <a href="http://www.numbeo.com/cost-of-living/prices_by_country.jsp?displayCurrency=EUR&itemId=105">
        http://www.numbeo.com/cost-of-living/prices_by_country.jsp?displayCurrency=EUR&itemId=105
        </a>
        </small>
        </div>
      </body>
    </html>
    """
    return page_template


def main():
    # Load data from CVS file
    afile = "TD_M.csv"
    datarows = []
    with open(afile, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # skip header
        for row in reader:
            datarows.append(row)

    # Describe data
    description = {"country": ("string", "Country"),
                       "dpi": ("number", "Total/million"), }

    # Build list of dictionaries from loaded data
    data = []
    for each in datarows:
        data.append({"country": each[0],
                     "dpi": (float(each[1]), each[1])})

    # Instantiate DataTable with structure defined in 'description'
    data_table = gviz_api.DataTable(description)

    # Load it into gviz_api.DataTable
    data_table.LoadData(data)

    # Creating a JSon string
    json = data_table.ToJSon(columns_order=("country", "dpi"),
                             order_by="country", )
    
    # Put JSON string into the template
    # and save to output.html
    with open('T_D_per_million.html', 'w') as out:
        out.write(get_page_template() % (json,))

if __name__ == '__main__':
    main()
